In [1]:
# Import necessary libraries
import json
import requests
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_absolute_error

In [2]:
# Fetch data from API
response = requests.get('https://min-api.cryptocompare.com/data/histoday?fsym=ETH&tsym=CAD&limit=1000') 
json.loads(response.content)['Data'] # get content

In [3]:
# Convert JSON response to DataFrame
df = pd.DataFrame(json.loads(response.content)['Data']) # convert to data frame
df.head()

In [4]:
# Set time as index
df = df.set_index('time')
df.head()

In [5]:
# Select relevant columns
df = df[['high', 'low', 'open', 'volumefrom', 'volumeto', 'close']]
df.head()

In [6]:
# Function to split data into train and test sets
def train_test_split(df):
    split_row = len(df) - int(0.1 * len(df)) # 80% data for train
    train_data = df.iloc[:split_row] 
    test_data = df.iloc[split_row:] # 20% data for test
    return train_data, test_data
train, test = train_test_split(df) # separate data into train and test


In [7]:
# Display length of test set
len(test)

In [8]:
# Plot data
fig, ax = plt.subplots(1, figsize=(12, 8)) # show data
ax.plot(train['close'], label='train_data')
ax.plot(test['close'], label='test_data')

In [9]:
# Function to normalize data (zero-base normalization)
def norm_zero_base(df):
    return df / df.iloc[0] - 1 # divide all numbers by the first number in the column

In [10]:
# Example of zero-base normalization
sample = norm_zero_base(df['close']) # show for example
fig, ax = plt.subplots(1, figsize=(12, 8))
ax.plot(sample, label='sample')

In [11]:
# Function to normalize data (min-max normalization)
def norm_min_max(df):
    return (df - df.min()) / (df.max() - df.min())

In [12]:
# Example of min-max normalization
sample = norm_min_max(df['close'])
fig, ax = plt.subplots(1, figsize=(12, 8))
ax.plot(sample, label='sample')

In [13]:
# Function to extract windowed data
def extract_windows_data(df):
    window_data = []
    for idx in range(len(df) - 10):
        tmp = df[idx: (idx + 10)].copy()
        tmp = norm_zero_base(tmp)
        window_data.append(tmp.values)
    return np.array(window_data)

In [14]:
# Function to prepare data for training
def prepare_data(df):
    train_data, test_data = train_test_split(df)
    X_train = extract_windows_data(train_data)
    X_test = extract_windows_data(test_data)
    y_train = train_data['close'][10:].values
    y_test = test_data['close'][10:].values
    
    y_train = y_train / train_data['close'][:-10].values - 1
    y_test = y_test / test_data['close'][:-10].values - 1
    return train_data, test_data, X_train, X_test, y_train, y_test

In [15]:
# Prepare data
train, test, X_train, X_test, y_train, y_test = prepare_data(df)

In [16]:
# Display y_train
y_train

In [17]:
# Display X_train
X_train[0]

In [18]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.5))
model.add(Dense(units=1))
model.add(Activation('relu'))

model.compile(loss='mse', optimizer='adam') # Mean Squared Error (MSE) loss function

In [19]:
# Train the model
history = model.fit(X_train, y_train, epochs=200, batch_size=32, shuffle=True, verbose=1)

In [20]:
# Make predictions on test set
targets = test['close'][10:]
preds = model.predict(X_test).squeeze()

# Calculate mean absolute error
mean_absolute_error(preds, y_test)

In [21]:
# Display predictions
preds

In [22]:
# Adjust predictions back to original scale
targets[:10] * (preds[:10] + 1)

In [23]:
# Predicted prices for the last 10 days of the training set
len(train[-10:]) * (preds[:10] + 1)

In [24]:
# Length of predictions
len(preds)

In [25]:
# Display predictions
preds

In [26]:
# Adjust predictions for plotting
preds_plt = preds.copy()
preds_plt[:60] = test['close'].iloc[:60].values * (preds_plt[:60] + 1)
preds_plt = pd.Series(index=targets.index, data=preds_plt)
fig, ax = plt.subplots(1, figsize=(12, 8))
ax.plot(targets, label='target')
ax.plot(preds_plt, label='predicted')
ax.legend()
plt.show()